In [3]:
from collections import defaultdict
from datetime import datetime
import math
from operator import itemgetter
import os
import random
import re

import numpy as np
import pandas as pd

%matplotlib inline

import little_mallet_wrapper as lmw

<br><br>

# MALLET Path

Change the path below to the location of your local MALLET installation.

In [4]:
path_to_mallet = 'C:/mallet/bin/mallet'  # CHANGE THIS TO YOUR MALLET PATH

# Load dataset

In [47]:
dataset_path = '../huawei_appdescrip_spider/3-LDA-应用-华为应用市场app分类与功能描述.csv'  # CHANGE THIS TO YOUR DATASET PATH

In [48]:
dataset_df = pd.read_csv(dataset_path,encoding="utf-8")
dataset_df.sample(5)

,分类,子分类,应用名,分词+去停用词+去低频词
13484,美食,菜谱,波波美食,波波 美食 详解 数 中华 料理 美味 厨 秘诀 令 享 居家 烹饪 乐 汇聚 国内 细致 ...
15517,出行导航,用车,百跑用车,百 跑 科技 旨在 利用 互联网 打通 电动 汽车 产业链 充电 服务 电动 汽车 租赁 全...
28675,金融理财,股票基金,容维财经,容维 财经 出品 款 轻便 简洁 专业 股票 分析 软件 提供 行情 股市 社区 基金 一站...
9644,教育,备考,2022初级会计考试必做习题,2022 初级 会计 考试 必 做 习题 题库 会计 从业 人员 考试 必 全新 真题 轻松...
36190,便捷生活,租房买房,搜房网买房,款 网罗 新房 二手房 租房 装修 专业 房产 软件 超过 2亿 买房 装修 用户 选择 下...


In [49]:
#index：第一列？除了首行
print(len(dataset_df.index))

38367


In [50]:
training_data = [t for t in dataset_df['分词+去停用词+去低频词'].tolist()]
training_data = [str(d).strip() for d in training_data if str(d)]

len(training_data)

38367

In [51]:
training_data[7]

'独播 影视 KOZNAK 加油 打工人 老师 就业 每日 挑战 职业 复合 独播 导师 面对 深陷 矛盾 嘉宾 化解 危机 办公室 趣事 咖 云集 合力 上演 办公室 爆笑 剧情 神仙 体验 明星 舞台 明星 做 发生 新奇 体验 父亲 愿望 父亲 成年 儿女 过程 擦 火花 快乐 追剧 KOZNAK 快乐 百分百 故事 丝绸之路 声 录音棚 声音 部 好剧 热播  劲爆 玩转 精彩 KOZNAK 积分 2.0 劲爆 来袭 首 次 签到 送 红包 分享 红包 分享 精彩 好友 好友 享受 欢乐 领取 红包 摇一摇 参与 摇一摇 VIP 会员 红包 羊 农场 快来 KOZNAK 领取 专属于 羊 会员 促销 每周五 会员 总 想不到 惊喜 温馨 提示 打开 客服 产品 路上 停歇 反馈 后续 改善 服务 微博 关注 抖音 关注 KOZNAK 微信 公众 号 KOZNAK 全新 来袭 全方位 提升 观影 感受 新剧 线 加油 打工人 首 播'

In [52]:
lmw.print_dataset_stats(training_data) #显示有关训练数据集的基本统计信息

Number of Documents: 38367
Mean Number of Words per Document: 92.0
Vocabulary Size: 49146


<br><br>

# Train topic model

Depending on the size of your dataset, training can take a while. For very large datasets, I recommending training outside of a notebook.

If you're training in a Jupyter notebook, you can observe progress by checking the terminal window from which you started Jupyter. The MALLET output will also be printed at the end of training.

Note that training creates a copy of your dataset, so it might require significant space.

## Training Option B: Train using import and training functions

Use these functions to have more control over the training parameters and output paths.

In [58]:
num_topics = 200  # CHANGE THIS TO YOUR PREFERRED NUMBER OF TOPICS

output_directory_path = 'huawei_LDA-output-' + str(num_topics) + '-2' # CHANGE THIS TO YOUR OUTPUT DIRECTORY

path_to_training_data           = output_directory_path + '/training.txt'
path_to_formatted_training_data = output_directory_path + '/mallet.training'
path_to_model                   = output_directory_path + '/mallet.model.' + str(num_topics)
path_to_topic_keys              = output_directory_path + '/mallet.topic_keys.' + str(num_topics)
path_to_topic_distributions     = output_directory_path + '/mallet.topic_distributions.' + str(num_topics)
path_to_word_weights            = output_directory_path + '/mallet.word_weights.' + str(num_topics)
path_to_diagnostics             = output_directory_path + '/mallet.diagnostics.' + str(num_topics) + '.xml'

    lmw.import_data：原始数据 -> txt格式数据 -> MALLET格式数据
  
  2：txt格式数据

  3: MALLET 格式数据

  4：原始数据

In [60]:

lmw.import_data(path_to_mallet,
                path_to_training_data,
                path_to_formatted_training_data,
                training_data)

Importing data...
Complete


使用 MALLET 训练 LDA 主题模型

In [61]:
lmw.train_topic_model(path_to_mallet,
                      path_to_formatted_training_data,
                      path_to_model,
                      path_to_topic_keys,
                      path_to_topic_distributions,
                      path_to_word_weights,
                      path_to_diagnostics,
                      num_topics)

Training topic model...
Complete


训练主题模型后，为每个主题加载最多的最可能单词集
每个主题最多20个词


In [62]:
# lmw.load_topic_keys(output_directory_path + '/mallet.topic_keys.' + str(num_topics))

topic_keys_path = output_directory_path + '/mallet.topic_keys.' + str(num_topics)
topic_keys = [line.split('\t')[2].split() for line in open(topic_keys_path, 'r',encoding="utf-8")] 

for i, t in enumerate(topic_keys):
    print(i, '\t', ' '.join(t))

0 	 有限公司 投资者 证监会 一站式 委员会 人民币 银行卡 互联网 管理人 全方位 子公司 基金业 身份证 许可证 全国性 第三方 收益率 客户端 管理局 非银行
1 	 关键词 UMU 全方位 供应商 服务业 平板车 随时随地 教育工作者 事务所 网页版 查询宝 安飞士 金融业 关键字 省时省力 来源于 主人翁 启信宝 工商局 阶段性
2 	 GRE GMAT 四六级 SAT 艾宾浩斯 记忆法 随时随地 词汇量 公开课 单词表 生词本 VOA 记不住 亲密无间 正确率 雅思托福 KMF 单词库 题例句 BBC
3 	 高颜值 随时随地 陌生人 年轻人 应有尽有 零距离 畅所欲言 面对面 脱口秀 排位赛 聊天室 邂逅心动 high 真人在线 与众不同 pick 眼花缭乱 不期而遇 短视频 聚集地
4 	 emoji photo stickers baby text Photo Editor cute background sticker add effects face Baby Bose editor make picture effect Face
5 	 倒计时 纪念日 计时器 节假日 备忘录 倒数日 课程表 计算器 信用卡 仪式感 工作日 时间表 亲朋好友 记事本 规划局 番茄钟 个性化 小部件 计划表 时间段
6 	 CAD 信贷员 DWG PDF dwg AutoCAD SView BIM 轻量化 信贷圈 DXF 源源不断 工程师 GitMind CATIA 测量员 找上门 兼容天正 XMind SolidWorks
7 	 万年历 节假日 老黄历 二十四 纪念日 黄道吉日 黄历宜忌 历黄历 全方位 黄历万 随时随地 日历万 历日历 紫外线 综合性 百宝箱 老黄历万 倒计时 查宜忌 周期性
8 	 优惠券 电影票 肯德基 吃喝玩乐 一站式 应有尽有 麦当劳 星巴克 衣食住行 全品类 超低价 VIP 必胜客 爱奇艺 加油站 奖励金 大礼包 唯品会 火车票 苏宁易购
9 	 石家庄 哈尔滨 乌鲁木齐 安全员 鄂尔多斯 秦皇岛 呼和浩特 马鞍山 连云港 驻马店 攀枝花 三门峡 自治州 佳木斯 帮邦行 SuperMap 思乐智 救援员 辩护律师 歪尾狗
10 	 吃喝玩乐 客户端 影响力 随时随地 小伙伴 衣食住行 信息港 新鲜事 综合性 倒计时 一应俱全

在训练主题模型后加载每个文档的主题分布

In [66]:
topic_distributions = lmw.load_topic_distributions(output_directory_path + '/mallet.topic_distributions.' + str(num_topics))

len(topic_distributions), len(topic_distributions[0])

(38367, 200)

In [67]:
assert(len(topic_distributions) == len(training_data))

获取属于topic_index指示的topic的，概率最高的doc

n：输出n个doc

topic_index：第几个topic

In [68]:
for p, d in lmw.get_top_docs(training_data, topic_distributions, topic_index=0, n=3):
    print(round(p, 4), d) #概率取小数点后4位
    print()

0.9721 上海 证券 股票 开户 上海 证券 自主 研发 股票 开户 软件 专注 提升 用户 开户 体验 开户 过程 无需 审核 快速 上海 证券 开户 上海 证券 带 无限 开户 享 快速 开户 开户 沪 深 A 股 步 到位 快速 办理 需 等待 新人 礼 新人 开户 尊享 约定 年化 收益率 % 新客 理财券 嗨币 积分 开户 送 6666 指 e 商城 积分 礼 开户 前 下载 开户  建议 WiFi 4G 5G 环境 。2. 身份证 有效期 二代 身份证 银行卡 银行卡 公司 上海 证券 有限 责任 公司 成立 2001年 5月 系 百联 集团 有限公司 国泰 君安 证券 股份 有限公司 上海 国际 集团 有限公司 上海 资产 有限公司 上海 城投 集团 有限公司 投资 控股 综合 类 证券 公司 公司 注册 资本 元 公司 拥有 期货 子公司 分公司 营业 网点 76 上海 长 三角 珠三角 京 津 冀 经济圈 主体 经营 公司 涵盖 场内 场外 市场 投资者 提供 证券 经纪 财富 投资 全方位 综合 服务 公司 成立 始终 秉承 诚信 专业 创新 务实 核心 价值观 效益 规模 结构 协调 发展 探索 创新 努力 打造 上海 证券 经营 品牌 资产 规模 持续 经营 实力 增强 社会效益 经济效益 展望 未来 公司 综合化 经营 差异化 竞争 战略 立足 普惠 产品 服务 指引 加大 互联网 实践 提升 客户 体验 竭力 投资者 提供 服务 股票 投资 软件 过程 公司 官网 微信 客服 热线 电子 信箱 开户 全新 升级

0.9672 成立 时间 英 基金 有限公司 简称 公司 成立 2012年 8月 中国 证监会 批准 成立 全国性 公募 基金 公司 涵盖 基金 募集 基金 销售 特定 客户 资产 资产 中国 证监会 许可 股东 介绍 公司 注册 资本金 元 国网 英 国际 控股 集团 有限公司 持 股 中国 交通 建设 股份 有限公司 持 股 航天 科工 财务 有限 责任 公司 持 股 下设 全资 子公司 北京 英 资本 有限公司 特定 客户 资产 法律 法规 中国 证监会 许可 公司 公司 始终 基金 持有人 利益 原则 倡导 价值 投资 稳健 投资 风格 诚信 责任 专业 创新 企业 精神 特色 公司 文化 公司 投研 团队 资本 市

加载主题词分布。这些是每个主题的每个单词的概率

In [64]:
# topic_word_probability_dict = lmw.load_topic_word_distributions(output_directory_path + '/mallet.word_weights.' + str(num_topics))

word_weight_path = output_directory_path + '/mallet.word_weights.' + str(num_topics)

topic_word_weight_dict = defaultdict(lambda: defaultdict(float))
topic_sum_dict = defaultdict(float)
with open(word_weight_path,'r',encoding="utf-8") as f:       
    for _line in f:        
        _topic, _word, _weight = _line.split('\t')
        topic_word_weight_dict[_topic][_word] = float(_weight)
        topic_sum_dict[_topic] += float(_weight)

topic_word_probability_dict = defaultdict(lambda: defaultdict(float))
for _topic, _word_weight_dict in topic_word_weight_dict.items():
    for _word, _weight in _word_weight_dict.items():
        topic_word_probability_dict[int(_topic)][_word] = _weight / topic_sum_dict[_topic]

len(topic_word_probability_dict)

200

In [65]:
for _topic, _word_probability_dict in topic_word_probability_dict.items():
    print('Topic', _topic)
    for _word, _probability in sorted(_word_probability_dict.items(), key=lambda x: x[1], reverse=True)[:5]:
        print(round(_probability, 4), '\t', _word)
    print()

Topic 0
0.1282 	 有限公司
0.0728 	 投资者
0.0389 	 证监会
0.0256 	 一站式
0.0229 	 委员会

Topic 1
0.0778 	 关键词
0.0413 	 UMU
0.0397 	 全方位
0.0334 	 供应商
0.0302 	 服务业

Topic 2
0.0766 	 GRE
0.0725 	 GMAT
0.0569 	 四六级
0.0271 	 SAT
0.0258 	 艾宾浩斯

Topic 3
0.1577 	 高颜值
0.0968 	 随时随地
0.0302 	 陌生人
0.0255 	 年轻人
0.0229 	 应有尽有

Topic 4
0.0495 	 emoji
0.0452 	 photo
0.0367 	 baby
0.0367 	 stickers
0.0319 	 text

Topic 5
0.2044 	 倒计时
0.191 	 纪念日
0.0466 	 计时器
0.04 	 节假日
0.0355 	 备忘录

Topic 6
0.2397 	 CAD
0.0805 	 信贷员
0.0406 	 DWG
0.0276 	 PDF
0.0252 	 dwg

Topic 7
0.1691 	 万年历
0.0901 	 节假日
0.0765 	 老黄历
0.0425 	 二十四
0.0332 	 纪念日

Topic 8
0.194 	 优惠券
0.0407 	 电影票
0.0402 	 肯德基
0.0302 	 吃喝玩乐
0.0281 	 一站式

Topic 9
0.0848 	 石家庄
0.0783 	 哈尔滨
0.0704 	 乌鲁木齐
0.0496 	 安全员
0.0417 	 鄂尔多斯

Topic 10
0.0969 	 吃喝玩乐
0.0793 	 客户端
0.0538 	 影响力
0.034 	 随时随地
0.0329 	 小伙伴

Topic 11
0.0749 	 人工智能
0.061 	 紫外线
0.0521 	 排行榜
0.044 	 全方位
0.0374 	 国内外

Topic 12
0.1357 	 加速器
0.04 	 LOL
0.0342 	 CSGO
0.0303 	 lol
0.0205 	 DOTA

Topic 13
0.0824 	 目的

- 计算两个目标主题分布之间的 Jensen-Shannon 散度

In [ ]:
lmw.get_js_divergence_topics(0, 7, topic_word_probability_dict)

# Plot topics by category

如果数据集包含一段时间的分类标签，则创建标签 x 主题的热图

In [ ]:
target_labels = ['John Keats', 'Emily Dickinson', 'William Butler Yeats', 'Christina Rossetti']

lmw.plot_categories_by_topics_heatmap(authors,
                                      topic_distributions,
                                      topic_keys, 
                                      output_directory_path + '/categories_by_topics.pdf',
                                      target_labels=target_labels,
                                      dim=(12,4))

如果数据集包含一些时间的分类标签，则创建一组箱线图，每个主题一个图。

In [ ]:
target_labels = ['John Keats', 'Emily Dickinson', 'William Butler Yeats', 'Christina Rossetti']

for _topic_index in range(0, len(topic_keys)):
    
    lmw.plot_categories_by_topic_boxplots(authors,
                                          topic_distributions,
                                          topic_keys, 
                                          _topic_index,
                                          output_path=output_directory_path + '/boxplot.' + str(_topic_index) + '.pdf',
                                          target_labels=target_labels,
                                          dim=(4,4))

<br><br>

# Plot topics over document time

Splits each training document into N chunks, infers the topics for those chunks (using the pre-trained topic model), and plots mean topic probabilities over document time.

In the example shown, it looks like poems often start with physical descriptions and end with abstract and romantic language.

给定一个数据集，将每个文档分成num_chunks个doc

In [ ]:
divided_documents, document_ids, times = lmw.divide_training_data(training_data,
                                                                  num_chunks=10)

len(divided_documents), len(document_ids), len(times)

In [ ]:
path_to_divided_training_data           = output_directory_path + '/training.split.txt'
path_to_divided_formatted_training_data = output_directory_path + '/mallet.split.training'
path_to_divided_topic_distributions     = output_directory_path + '/mallet.split.topic_distributions.' + str(num_topics)

    lmw.import_data：原始数据 -> txt格式数据 -> MALLET格式数据
  
  2：txt格式数据

  3: MALLET 格式数据

  4：原始数据

In [ ]:
lmw.import_data(path_to_mallet,
                path_to_divided_training_data,
                path_to_divided_formatted_training_data,
                divided_documents,
                use_pipe_from=path_to_formatted_training_data)

# **模型使用**
使用已在另一组文档上训练的模型获取一组新文档的主题分布

In [ ]:
lmw.infer_topics(path_to_mallet,
                 path_to_model,
                 path_to_divided_formatted_training_data,
                 path_to_divided_topic_distributions)

在训练主题模型后加载每个文档的主题分布

In [ ]:
topic_distributions = lmw.load_topic_distributions(path_to_divided_topic_distributions)

len(topic_distributions), len(topic_distributions[2])

创建线图，每个主题一个，显示文档段的平均主题概率

In [ ]:
for _topic_index in range(0, len(topic_keys)):
    lmw.plot_topics_over_time(topic_distributions, topic_keys, times, _topic_index)